In [1]:
import pandas as pd
from pandas import json_normalize
import requests
from bs4 import BeautifulSoup
from random_user_agent.user_agent import UserAgent
import pyarrow as pa
import pyarrow.parquet as pq
import re
import os
import concurrent.futures
import asyncio
import aiohttp
import datetime, pytz
from utils.utils_scraper import ScraperArgenProp
import unidecode
from utils.lat_long import get_geocoding, apply_geocoding, get_state
from geopy.geocoders import Nominatim
import concurrent.futures
import geocoder
import time

In [3]:
url_zona = 'https://www.zonaprop.com.ar/departamentos-alquiler-rosario-pagina-1.html'

# Argenprop

In [2]:
# Definindo user agent aleatórios e headers da requisição
tipos = ['campos','casas','cocheras','departamentos','fondos-de-comercio','galpones','hoteles','locales','negocios-especiales','oficinas','ph','quintas','terrenos']

# url_argen = f'https://www.argenprop.com/{tipo}/alquiler/{local}?pagina-{pagina}'
url_argen = 'https://www.argenprop.com/departamentos/alquiler/rosario?pagina-1'

ua = UserAgent()
user_agents = ua.get_random_user_agent()
headers = {'user-agent': user_agents.strip(), 'encoding':'utf-8'}

r = requests.get(url_argen, headers = headers)

soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
# TOTAL DE IMOVEIS DA PAGINA

imoveis = float(re.sub('[^0-9]','', soup.find('p','listing-header__results').text))
imoveis

543.0

In [2]:
# max_page[(max_page['tipo'] == 'departamentos') & (max_page['local'] == 'rosario')]['paginas'].max()
_tipo = ['campos','casas','cocheras','departamentos','fondos-de-comercio','galpones','hoteles','locales','negocios-especiales','oficinas','ph','quintas','terrenos']
_local = ['rosario']
page_df = await ScraperArgenProp(_tipo = _tipo, _local = _local).get_property_data()

[2024-01-08 21:12:54] - INFO - get_property_data - Obtendo lista de htmls para extrair dados de imóveis!
[2024-01-08 21:12:54] - INFO - total_pages - Obtendo lista de htmls para definir total de páginas por tipo de imóvel!
[2024-01-08 21:12:54] - INFO - total_pages - Iniciando execução assíncrona das tasks
[2024-01-08 21:12:54] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/galpones/alquiler/rosario?pagina-1
[2024-01-08 21:12:56] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/locales/alquiler/rosario?pagina-1
[2024-01-08 21:12:56] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/hoteles/alquiler/rosario?pagina-1
[2024-01-08 21:12:56] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/ph/alquiler/rosario?pagina-1
[2024-01-08 21:12:56] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/casas/alquiler/rosario?pagina-1
[2024-01-08 21:12:56] - INFO - get_page - Obtendo cód

In [17]:
page_df.columns

Index(['id', 'base', 'tipo_imovel', 'estado', 'cidade', 'bairro', 'endereco',
       'url', 'descricao', 'titulo', 'aluguel_moeda', 'aluguel_valor',
       'expensas', 'area', 'antiguidade', 'banheiros', 'tipo_banheiro',
       'ambientes', 'dormitorios', 'orientacao', 'garagens',
       'estado_propriedade', 'tipo_local', 'imobiliaria', 'num_fotos', 'fotos',
       'card_points', 'wsp', 'data', 'ano', 'mes', 'dia'],
      dtype='object')

In [19]:
(
    page_df
    .loc[page_df.tipo_imovel == 'departamentos']
    .groupby(by=['tipo_imovel','aluguel_moeda','dormitorios'])
    .agg(
            aluguel = ('aluguel_valor', 'median'),
            imoveis = ('id','count')
    )
    .round(2)
)

aluguel  imoveis
tipo_imovel   aluguel_moeda dormitorios                   
departamentos $             0.0          130000.0      129
                            1.0          160000.0      262
                            2.0          240000.0      133
                            3.0          290000.0       31
                            4.0          370000.0        6
              Sem info      0.0               0.0        5
                            1.0               0.0       10
                            2.0               0.0        3
                            3.0               0.0        2
              USD           0.0             700.0        3
                            1.0             950.0        6
                            2.0             600.0       10
                            3.0             900.0        3
                            4.0            1650.0        2

In [28]:
_tipo = ['campos','casas','cocheras','departamentos','fondos-de-comercio','galpones','hoteles','locales','negocios-especiales','oficinas','ph','quintas','terrenos']
_local = ['rosario']

# max_page = pd.read_parquet(os.path.join(os.getcwd(),'data','imoveis','paginas','argenprop','bronze'))
page_df = ScraperArgenProp(_tipo = _tipo, _local = _local).page_data(paginas = 1)

urls = [
    f'https://www.argenprop.com/{tipo}/alquiler/{local}?pagina-{paginas}'
    for tipo in _tipo
    for local in _local
    for max_pages in [int(max_page[(max_page['tipo'] == tipo) & (max_page['local'] == local)]['paginas'].max())]
    for paginas in range(1, max_pages + 1)
]

In [9]:
tipos = ['campos','casas','cocheras','departamentos','fondos-de-comercio','galpones','hoteles','locales','negocios-especiales','oficinas','ph','quintas','terrenos']
# for tipo in tipos:
#     print(paginas('rosario-ros', tipo))

with concurrent.futures.ThreadPoolExecutor(max_workers = 2) as executor:
    # Criando a sequência de tasks que serão submetidas para a thread pool
    urls = {executor.submit(paginas, local, tipo): local for local in ['rosario-ros'] for tipo in ['campos','casas','cocheras','departamentos','fondos-de-comercio','galpones','hoteles','locales','negocios-especiales','oficinas','ph','quintas','terrenos']}
    
    for future in concurrent.futures.as_completed(urls):
        url = urls[future]
        try:
            resultado = future.result()
            print(resultado)
        except Exception as exc:
            print(f'Erro na operação: {exc}')

{'local': 'rosario-ros', 'tipo': 'campos', 'imoveis': None}
{'local': 'rosario-ros', 'tipo': 'casas', 'imoveis': 129.0}
{'local': 'rosario-ros', 'tipo': 'cocheras', 'imoveis': 26.0}
{'local': 'rosario-ros', 'tipo': 'departamentos', 'imoveis': 533.0}
{'local': 'rosario-ros', 'tipo': 'fondos-de-comercio', 'imoveis': 1.0}
{'local': 'rosario-ros', 'tipo': 'galpones', 'imoveis': 79.0}
{'local': 'rosario-ros', 'tipo': 'locales', 'imoveis': 213.0}
{'local': 'rosario-ros', 'tipo': 'hoteles', 'imoveis': None}
{'local': 'rosario-ros', 'tipo': 'negocios-especiales', 'imoveis': 6.0}
{'local': 'rosario-ros', 'tipo': 'oficinas', 'imoveis': 85.0}
{'local': 'rosario-ros', 'tipo': 'ph', 'imoveis': 14.0}
{'local': 'rosario-ros', 'tipo': 'quintas', 'imoveis': None}
{'local': 'rosario-ros', 'tipo': 'terrenos', 'imoveis': 8.0}


In [38]:
dados = []

for x in page_df:
    try:
        soup = BeautifulSoup(x['html'], 'html.parser')

        res = soup.find('div', {"class":"listing__items"})

        for i in res:
            # CIDADE
            try:
                # cidade = re.search(r',\s*([^,]+)', i.find('p','card__title--primary show-mobile').text).group(1).replace('Departamento de','').strip()
                cidade = x['local']
            except:
                cidade = 'Sem info'

            # TIPO IMOVEL
            try:
                # cidade = re.search(r',\s*([^,]+)', i.find('p','card__title--primary show-mobile').text).group(1).replace('Departamento de','').strip()
                tipo_imo = x['tipo']
            except:
                tipo_imo = 'Sem info'

            # BASE DOS DADOS
            try:
                # cidade = re.search(r',\s*([^,]+)', i.find('p','card__title--primary show-mobile').text).group(1).replace('Departamento de','').strip()
                base = x['base']
            except:
                base = 'Sem info'

            # DATA
            data = x['data']
            ano = x['ano']
            mes = x['mes']
            dia = x['dia']

            # ID DO IMOVEL - OK
            try:
                id = i.find('a').get('data-item-card')
            except:
                id = 'Sem info'

            # URL - OK
            try:
                url = 'https://www.argenprop.com' + i.find('a').get('href')
            except:
                url = 'Sem info'

            # NUMERO DE FOTOS - OK
            try:
                num_fotos = int(re.search(r'(?<=/)\d+', i.find('div','counter-box').text).group(0))
            except:
                num_fotos = 0.0

            # ENDERECO - OK
            try:
                endereco = (i.find('p', 'card__address').text).strip().lower()
            except:
                endereco = 'Sem info'

            # BAIRRO - OK
            try:
                bairro = re.match(r'^([^,]+)', (i.find('p','card__title--primary show-mobile').text).strip()).group(0)
            except:
                bairro = 'Sem info'

            # ALUGUEL - MOEDA
            try:
                aluguel_moeda = i.find('span','card__currency').text
            except:
                aluguel_moeda = 'Sem info'
            
            # ALUGUEL - VALOR
            try:
                aluguel = float(re.sub(r'[^0-9]', '', i.find('span','card__currency').next_sibling.strip()))
            except:
                aluguel = 0.0

            # EXPENSAS
            try:
                expensas = float(re.sub(r'[^0-9]', '', i.find('span','card__expenses').text.strip()))
            except:
                expensas = 0.0

            # IMOBILIARIA
            try:
                imobiliaria = i.find('div', 'card__agent').find('img').get('alt').strip().lower()
            except:
                imobiliaria = 'Sem info'

            # FOTOS
            try:
                fotos = [p.find('img').get('data-src') for p in i.find('ul', 'card__photos').find_all('li')]
            except:
                fotos = 'Sem info'

            # CARD POINTS - DEFINE A ORDEM EM QUE OS IMOVEIS APARECEM NO SITE (APARENTEMENTE)
            try:
                card_points = float(re.sub('[^0-9]','',i.find('p','card__points').text.strip()))
            except:
                card_points = 0.0

            # DESCRICAO
            try:
                descricao = i.find('p', 'card__info').text.strip()
            except:
                descricao = 'Sem info'
            
            # TITULO
            try:
                titulo = i.find('h2', 'card__title').text.strip()
            except:
                titulo = 'Sem info'

            # AMENIDADES
            try:
                # Lista com todas as amenidades para facilitar extração das existentes
                amenidades = i.find('ul', 'card__main-features').find_all('li')
            except:
                amenidades = None

            # AREA
            try:
                area_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-superficie_cubierta') != None]
                area = float(re.match('[0-9]{1,}', amenidades[area_indice[0]].find('span').text.strip()).group(0))
            except:
                area = 0.0

            # ANTIGUIDADE
            try:
                antiguidade_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-antiguedad') != None]
                antiguidade = float(re.match('[0-9]{1,}', amenidades[antiguidade_indice[0]].find('span').text.strip()).group(0)) if 'estrenar' not in amenidades[antiguidade_indice[0]].find('span').text.strip().lower() else 0.0
            except:
                antiguidade = 0.0

            # BANHEIROS
            try:
                banheiros_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-cantidad_banos') != None]
                banheiros = float(re.match('[0-9]{1,}', amenidades[banheiros_indice[0]].find('span').text.strip()).group(0))
            except:
                banheiros = 0.0

            # TIPO BANHEIROS
            try:
                tipo_banheiro_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-tipo_baño') != None]
                tipo_banheiro = amenidades[tipo_banheiro_indice[0]].find('span').text.strip()
            except:
                tipo_banheiro = 'Sem info'

            # AMBIENTES
            try:
                ambientes_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-cantidad_ambientes') != None]
                ambientes = amenidades[ambientes_indice[0]].find('span').text.strip()
            except:
                ambientes = 'Sem info'
            
            # DORMITORIOS
            try:
                dormitorios_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-cantidad_dormitorios') != None]
                dormitorios = float(re.match('[0-9]{1,}', amenidades[dormitorios_indice[0]].find('span').text.strip()).group(0))
            except:
                dormitorios = 0.0

            # ORIENTACAO
            try:
                orientacao_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-orientacion') != None]
                orientacao = amenidades[orientacao_indice[0]].find('span').text.strip()
            except:
                orientacao = 'Sem info'

            # GARAGENS
            try:
                garagens_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-ambiente_cochera') != None]
                garagens = float(re.match('[0-9]{1,}', amenidades[garagens_indice[0]].find('span').text.strip()).group(0))
            except:
                garagens = 0.0
            
            # ESTADO PROPRIEDADE
            try:
                estado_prop_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-estado_propiedad') != None]
                estado_prop = amenidades[estado_prop_indice[0]].find('span').text.strip()
            except:
                estado_prop = 'Sem info'
            
            # TIPO DE LOCAL
            try:
                tipo_local_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-tipo_local') != None]
                tipo_local = amenidades[tipo_local_indice[0]].find('span').text.strip()
                # print(tipo_local)
            except:
                tipo_local = 'Sem info'

            # WHATSAPP
            try:
                wsp = re.findall(r'https://wa\.me/\d+', i.find('div', 'card-contact-group').find('span')['data-href'])[0]
            except:
                wsp = 'Sem info'

            # Adicionando à lista
            dados.append(
                [
                    id,
                    base,
                    tipo_imo,
                    cidade, 
                    bairro,
                    endereco,
                    url, 
                    descricao,
                    titulo,
                    aluguel_moeda,
                    aluguel,
                    expensas,
                    area,
                    antiguidade, 
                    banheiros,
                    tipo_banheiro,
                    ambientes,
                    dormitorios,
                    orientacao,
                    garagens,
                    estado_prop,
                    tipo_local, 
                    imobiliaria, 
                    num_fotos,
                    fotos,
                    card_points,
                    wsp,
                    data,
                    ano,
                    mes,
                    dia
                ]
            )
    except:
        continue

In [49]:
df = pd.DataFrame(
    dados, 
    columns = [
        'id',
        'base',
        'tipo_imovel',
        'cidade', 
        'bairro',
        'endereco',
        'url', 
        'descricao',
        'titulo',
        'aluguel_moeda',
        'aluguel_valor',
        'expensas',
        'area',
        'antiguidade', 
        'banheiros',
        'tipo_banheiro',
        'ambientes',
        'dormitorios',
        'orientacao',
        'garagens',
        'estado_propriedade',
        'tipo_local', 
        'imobiliaria', 
        'num_fotos',
        'fotos',
        'card_points',
        'wsp',
        'data',
        'ano',
        'mes',
        'dia'
    ]
)
df = (
    df
    .drop(index = df[(df['id'].isnull()) | (df['id'] == 'Sem info')].index)
    .drop_duplicates(subset = ['id','cidade','tipo_imovel'])
    .sort_values(by = ['cidade','tipo_imovel','bairro'])
    .reset_index(drop = True)
)

In [50]:
df

,id,base,tipo_imovel,cidade,bairro,endereco,url,descricao,titulo,aluguel_moeda,...,tipo_local,imobiliaria,num_fotos,fotos,card_points,wsp,data,ano,mes,dia
0,12671111,argenprop,campos,rosario,Alvear,autopista buenos aires km al 283,https://www.argenprop.com/campo-en-alquiler-en...,Se alquilan 6 hectáreas juntas o fraccionadas ...,Campo - Alvear,$,...,Sem info,domum negocios inmobiliarios,9.0,[https://static1.sosiva451.com/11117621/dd79eb...,0.0,https://wa.me/5493416422361,2023-12-09 17:35:38,2023,12,9
1,10913192,argenprop,campos,rosario,Perez,ruta 100,https://www.argenprop.com/campo-en-alquiler-en...,En alquiler fraccion de campo en perez co...,Excelente fracción en zona industrial de Pérez...,$,...,Sem info,casa lazio bienes raices,2.0,[https://static1.sosiva451.com/29131901/e36941...,0.0,Sem info,2023-12-09 17:35:38,2023,12,9
2,14890819,argenprop,casas,rosario,Abasto,ocampo y 1 de mayo rosario,https://www.argenprop.com/casa-en-alquiler-en-...,"Impecable casa duplex 3 dormitorios al frente,...",Casa duplex 3 dorm en alquiler en Abasto Rosario,$,...,Sem info,Sem info,2.0,[https://static1.sosiva451.com/91809841/f13451...,1006.0,https://wa.me/54341153602022,2023-12-09 17:35:38,2023,12,9
3,13238831,argenprop,casas,rosario,Abasto,san martìn al 2200,https://www.argenprop.com/casa-en-alquiler-en-...,La propiedad se ofrece solo para destino comer...,Alquiler - Casa 4 dormitorios (uso comercial) ...,$,...,Sem info,echeverria - massoud,47.0,[https://static1.sosiva451.com/13883231/740287...,115.0,https://wa.me/5493416116361,2023-12-09 17:35:38,2023,12,9
4,14893136,argenprop,casas,rosario,Abasto,juan manuel de rosas 2200,https://www.argenprop.com/casa-en-alquiler-en-...,Hermosa casa estilo chalet en alquiler en una ...,Casa en alquiler de tres dormitorios,$,...,Sem info,saavedra tobajas propiedades,16.0,[https://static1.sosiva451.com/63139841/84be32...,9.0,https://wa.me/5493413102345,2023-12-09 17:35:43,2023,12,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,12048187,argenprop,terrenos,rosario,Rosario,"rn 9 km 270, pueblo esther",https://www.argenprop.com/terreno-en-alquiler-...,Campo de 4 hectáreas\n \n - ...,Terreno baldió en alquiler - Pueblo Esther,$,...,Sem info,grupo ambito srl,3.0,[https://static1.sosiva451.com/78184021/a3c460...,0.0,https://wa.me/5493412014444,2023-12-09 17:35:38,2023,12,9
1253,12956342,argenprop,terrenos,rosario,Rosario,batle y ordoñez y españa 1700,https://www.argenprop.com/terreno-en-alquiler-...,Terreno ubicado en batlle y ordoñez (sur) entr...,Terreno ubicado en Batlle y Ordoñez y España,Sem info,...,Sem info,remax profesional,1.0,[https://static1.sosiva451.com/24365921/8dde66...,0.0,Sem info,2023-12-09 17:35:38,2023,12,9
1254,13923771,argenprop,terrenos,rosario,Rosario,autopista n9 y gallegos s/n,https://www.argenprop.com/terreno-en-alquiler-...,Alquiler - terreno sobre autopista rosario - c...,Alquiler Terreno AU ROS/COR - Ideal industrial,Sem info,...,Sem info,remax exclusivo,4.0,[https://static1.sosiva451.com/17732931/5f762f...,0.0,Sem info,2023-12-09 17:35:38,2023,12,9
1255,14444871,argenprop,terrenos,rosario,Rosario,santiago al 5600,https://www.argenprop.com/terreno-en-alquiler-...,Ajuste trimestral x ipc\n\n\nsantiago casi ave...,SANTIAGO AL 5600 - ALQUILER DE GALPON EN ROSARIO,$,...,Sem info,dunod sa,18.0,[https://static1.sosiva451.com/17844441/a0d9ad...,0.0,https://wa.me/5493416499764,2023-12-09 17:35:38,2023,12,9


In [147]:

# INFOS E AMENIDADES
res = soup.find('div', {"class":"listing__items"})

for i in res:
    # ID DO IMOVEL - OK
    try:
        id = i.find('a').get('data-item-card')
    except:
        id = 'Sem info'

    # URL - OK
    try:
        url = 'https://www.argenprop.com' + i.find('a').get('href')
    except:
        url = 'Sem info'

    # NUMERO DE FOTOS - OK
    try:
        num_fotos = int(re.search(r'(?<=/)\d+', i.find('div','counter-box').text).group(0))
    except:
        num_fotos = None

    # ENDERECO - OK
    try:
        endereco = (i.find('p', 'card__address').text).strip().lower()
    except:
        endereco = 'Sem info'

    # CIDADE
    try:
        cidade = re.search(r',\s*([^,]+)', i.find('p','card__title--primary show-mobile').text).group(1).replace('Departamento de','').strip()
    except:
        cidade = 'Sem info'

    # BAIRRO - OK
    try:
        bairro = re.match(r'^([^,]+)', (i.find('p','card__title--primary show-mobile').text).strip()).group(0)
    except:
        bairro = 'Sem info'

    # ALUGUEL - MOEDA
    try:
        aluguel_moeda = i.find('span','card__currency').text
    except:
        aluguel_moeda = None
    
    # ALUGUEL - VALOR
    try:
        aluguel = float(re.sub(r'[^0-9]', '', i.find('span','card__currency').next_sibling.strip()))
    except:
        aluguel = None

    # EXPENSAS
    try:
        expensas = float(re.sub(r'[^0-9]', '', i.find('span','card__expenses').text.strip()))
    except:
        expensas = None

    # IMOBILIARIA
    try:
        imobiliaria = i.find('div', 'card__agent').find('img').get('alt').strip().lower()
    except:
        imobiliaria = 'Sem info'

    # FOTOS
    try:
        fotos = [p.find('img').get('data-src') for p in i.find('ul', 'card__photos').find_all('li')]
    except:
        fotos = None

    # CARD POINTS - DEFINE A ORDEM EM QUE OS IMOVEIS APARECEM NO SITE (APARENTEMENTE)
    try:
        card_points = float(re.sub('[^0-9]','',i.find('p','card__points').text.strip()))
    except:
        card_points = None

    # DESCRICAO
    try:
        descricao = i.find('p', 'card__info').text.strip()
    except:
        descricao = 'Sem info'
    
    # TITULO
    try:
        titulo = i.find('h2', 'card__title').text.strip()
    except:
        titulo = 'Sem info'

    # AMENIDADES
    try:
        # Lista com todas as amenidades para facilitar extração das existentes
        amenidades = i.find('ul', 'card__main-features').find_all('li')
    except:
        amenidades = None

    # AREA
    try:
        area_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-superficie_cubierta') != None]
        area = float(re.match('[0-9]{1,}', amenidades[area_indice[0]].find('span').text.strip()).group(0))
    except:
        area = None

    # ANTIGUIDADE
    try:
        antiguidade_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-antiguedad') != None]
        antiguidade = float(re.match('[0-9]{1,}', amenidades[antiguidade_indice[0]].find('span').text.strip()).group(0)) if 'estrenar' not in amenidades[antiguidade_indice[0]].find('span').text.strip().lower() else 0.0
    except:
        antiguidade = None

    # BANHEIROS
    try:
        banheiros_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-cantidad_banos') != None]
        banheiros = float(re.match('[0-9]{1,}', amenidades[banheiros_indice[0]].find('span').text.strip()).group(0))
    except:
        banheiros = None

    # TIPO BANHEIROS
    try:
        tipo_banheiro_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-tipo_baño') != None]
        tipo_banheiro = amenidades[tipo_banheiro_indice[0]].find('span').text.strip()
    except:
        tipo_banheiro = None

    # AMBIENTES
    try:
        ambientes_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-cantidad_ambientes') != None]
        ambientes = amenidades[ambientes_indice[0]].find('span').text.strip()
    except:
        ambientes = None
    
    # DORMITORIOS
    try:
        dormitorios_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-cantidad_dormitorios') != None]
        dormitorios = float(re.match('[0-9]{1,}', amenidades[dormitorios_indice[0]].find('span').text.strip()).group(0))
    except:
        dormitorios = None

    # ORIENTACAO
    try:
        orientacao_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-orientacion') != None]
        orientacao = amenidades[orientacao_indice[0]].find('span').text.strip()
    except:
        orientacao = None

    # GARAGENS
    try:
        garagens_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-ambiente_cochera') != None]
        garagens = float(re.match('[0-9]{1,}', amenidades[garagens_indice[0]].find('span').text.strip()).group(0))
    except:
        garagens = None
    
    # ESTADO PROPRIEDADE
    try:
        estado_prop_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-estado_propiedad') != None]
        estado_prop = amenidades[estado_prop_indice[0]].find('span').text.strip()
    except:
        estado_prop = 'Sem info'
    
    # TIPO DE LOCAL
    try:
        tipo_local_indice = [i for i, s in enumerate(amenidades) if s.find('i','icono-tipo_local') != None]
        tipo_local = amenidades[tipo_local_indice[0]].find('span').text.strip()
        # print(tipo_local)
    except:
        tipo_local = 'Sem info'

    # WHATSAPP
    try:
        wsp = re.findall(r'https://wa\.me/\d+', i.find('div', 'card-contact-group').find('span')['data-href'])[0]
    except:
        wsp = None

Comunes


# Dados Geográficos Argentina

In [46]:
df_final = (
    pd.merge(
        left = df_muni,
        right = df_prov,
        how = 'left',
        left_on = ['id_provincia'],
        right_on = ['id']
    )
    .drop('id', axis = 1)
    .assign(municipio_tratado = lambda df: df['municipio'].apply(lambda x: unidecode.unidecode(str(re.sub(' +', ' ', x.strip())).lower())))
    .assign(provincia_tratada = lambda df: df['provincia'].apply(lambda x: unidecode.unidecode(str(re.sub(' +', ' ', x.strip())).lower())))
    .rename(
        columns ={
            'latitude_x': 'latitude_municipio',
            'longitude_x': 'longitude_municipio',
            'latitude_y': 'latitude_provincia',
            'longitude_y': 'longitude_provincia',
        }
    )
    [['id_municipio', 'municipio', 'latitude_municipio', 'longitude_municipio', 'id_provincia', 'provincia', 'latitude_provincia', 'longitude_provincia', 'municipios_provincia', 'municipio_tratado', 'provincia_tratada']]
)

In [5]:
df_2 = pd.read_parquet(
    os.path.join(os.getcwd(),'data','geograficos','provincias_municipios') if os.getcwd().__contains__('app') else os.path.join(os.getcwd(),'app','data','geograficos','provincias_municipios')
)
df_

,id_municipio,municipio,latitude_municipio,longitude_municipio,id_provincia,provincia,latitude_provincia,longitude_provincia,municipios_provincia,municipio_tratado,provincia_tratada
0,060056,Bahía Blanca,-38.580855,-62.165690,06,Buenos Aires,-36.676942,-60.558832,135,bahia blanca,buenos aires
1,060280,General Alvarado,-38.202713,-58.072249,06,Buenos Aires,-36.676942,-60.558832,135,general alvarado,buenos aires
2,060357,General Pueyrredón,-37.965676,-57.743144,06,Buenos Aires,-36.676942,-60.558832,135,general pueyrredon,buenos aires
3,060518,Mar Chiquita,-37.498434,-57.643618,06,Buenos Aires,-36.676942,-60.558832,135,mar chiquita,buenos aires
4,060644,Pinamar,-37.109946,-56.869675,06,Buenos Aires,-36.676942,-60.558832,135,pinamar,buenos aires
...,...,...,...,...,...,...,...,...,...,...,...
1809,900042,Municipalidad de Famailla,-26.978707,-65.472459,90,Tucumán,-26.947800,-65.364758,112,municipalidad de famailla,tucuman
1810,908357,El Manantial,-26.839531,-65.322264,90,Tucumán,-26.947800,-65.364758,112,el manantial,tucuman
1811,940007,Río Grande,-53.858902,-67.798434,94,"Tierra del Fuego, Antártida e Islas del Atlánt...",-82.521518,-50.742749,3,rio grande,"tierra del fuego, antartida e islas del atlant..."
1812,942007,Tolhuin,-54.576728,-67.195800,94,"Tierra del Fuego, Antártida e Islas del Atlánt...",-82.521518,-50.742749,3,tolhuin,"tierra del fuego, antartida e islas del atlant..."


In [24]:
df_dpt = (
    pd.read_excel(r'G:\Otros ordenadores\Meu modelo Laptop\Ciência de Dados\8. Projetos\web_scraping_arg\departamentos_argentina.xlsx')
    .assign(Nombre = lambda x: x['Nombre'].str.lower().apply(lambda y: unidecode.unidecode(y)))
    .assign(Provincia = lambda x: x['Provincia'].str.lower().apply(lambda y: unidecode.unidecode(y)))
    .rename(
        columns = {
            'Nombre':'departamento',
            'Latitud':'latitude', 
            'Longitud':'longitude',
            'Provincia':'provincia'
        }
    )
    [['departamento','latitude', 'longitude','provincia']]
)

g:\Otros ordenadores\Meu modelo Laptop\Ciência de Dados\8. Projetos\web_scraping_arg\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [25]:
df_muni = pd.read_parquet(
    os.path.join(os.getcwd(),'data','geograficos','municipios') if os.getcwd().__contains__('app') else os.path.join(os.getcwd(),'app','data','geograficos','municipios')
)

df_prov = pd.read_parquet(
    os.path.join(os.getcwd(),'data','geograficos','departamentos') if os.getcwd().__contains__('app') else os.path.join(os.getcwd(),'app','data','geograficos','departamentos')
)

In [27]:
df_prov

,id,provincia,latitude,longitude
0,02,Ciudad Autónoma de Buenos Aires,-34.614493,-58.445856
1,06,Buenos Aires,-36.676942,-60.558832
2,10,Catamarca,-27.335833,-66.947682
3,14,Córdoba,-32.142933,-63.801753
4,18,Corrientes,-28.774305,-57.801219
5,22,Chaco,-26.386431,-60.765831
6,26,Chubut,-43.788623,-68.526759
7,30,Entre Ríos,-32.058874,-59.201448
8,34,Formosa,-24.894973,-59.932441
9,38,Jujuy,-23.320078,-65.764252


In [35]:
df = (
        pd.merge(
            left = df_dpt,
            right = df_prov.assign(provincia = lambda x: x['provincia'].str.lower().apply(lambda y: unidecode.unidecode(y))),
            how = 'left',
            left_on = 'provincia',
            right_on = 'provincia'
        )
        .rename(
            columns = {
                'latitude_x': 'latitude_municipio',		
                'longitude_x': 'longitude_municipio',
                'id': 'id_provincia',	
                'latitude_y': 'latitude_provincia',	
                'longitude_y': 'longitude_provincia'
            }
        )
    )


In [37]:
df

,departamento,latitude_municipio,longitude_municipio,provincia,id_provincia,latitude_provincia,longitude_provincia
0,san jeronimo,-32.239935,-61.231266,santa fe,82,-30.706927,-60.949837
1,yavi,-22.329812,-65.825982,jujuy,38,-23.320078,-65.764252
2,el cuy,-39.866718,-68.703438,rio negro,62,-40.405796,-67.229330
3,pergamino,-33.589305,-60.772765,buenos aires,06,-36.676942,-60.558832
4,villa gesell,-37.336673,-57.032501,buenos aires,06,-36.676942,-60.558832
...,...,...,...,...,...,...,...
525,san javier,-29.424207,-59.979555,santa fe,82,-30.706927,-60.949837
526,maipu,-37.016145,-57.488489,buenos aires,06,-36.676942,-60.558832
527,leandro n alem,-34.444846,-61.319236,buenos aires,06,-36.676942,-60.558832
528,comuna 14,-34.573900,-58.422436,ciudad de buenos aires,NaN,NaN,NaN


In [39]:
df_prov = pd.read_parquet(
    os.path.join(os.getcwd(),'data','geograficos','departamentos') if os.getcwd().__contains__('app') else os.path.join(os.getcwd(),'app','data','geograficos','departamentos')
)

In [42]:
cidade = 'Rosário'
estado = 'Santa Fé'
pais = 'Argentina'
cidade, estado, pais = unidecode.unidecode(cidade).lower(), unidecode.unidecode(estado).lower(), unidecode.unidecode(pais).lower()

print(cidade,estado,pais)

rosario santa fe argentina


In [65]:
df_dpt = pd.read_parquet(
        os.path.join(os.getcwd(),'data','geograficos','departamentos') if os.getcwd().__contains__('app') else os.path.join(os.getcwd(),'app','data','geograficos','departamentos')
    )

df_prov_muni = pd.read_parquet(
    os.path.join(os.getcwd(),'data','geograficos','provincias_municipios') if os.getcwd().__contains__('app') else os.path.join(os.getcwd(),'app','data','geograficos','provincias_municipios')
)

# Procurando a cidade nas bases
base_muni = df_prov_muni[df_prov_muni['municipio'] == cidade][['municipio','provincia']]
base_dpt = df_dpt.loc[df_dpt['departamento'] == cidade][['departamento','provincia']]

# Latitude/Longitude

In [11]:
_tipo = ['campos','casas','cocheras','departamentos','fondos-de-comercio','galpones','hoteles','locales','negocios-especiales','oficinas','ph','quintas','terrenos']
_local = ['rosario']

html = await ScraperArgenProp(_tipo = _tipo, _local = _local).get_all_pages()

[2023-12-17 11:48:40] - INFO - total_pages - Obtendo lista de htmls para definir total de páginas por tipo de imóvel!
[2023-12-17 11:48:40] - INFO - total_pages - Iniciando execução assíncrona das tasks
[2023-12-17 11:48:40] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/hoteles/alquiler/rosario?pagina-1
[2023-12-17 11:48:40] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/negocios-especiales/alquiler/rosario?pagina-1
[2023-12-17 11:48:40] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/cocheras/alquiler/rosario?pagina-1
[2023-12-17 11:48:41] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/galpones/alquiler/rosario?pagina-1
[2023-12-17 11:48:41] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/fondos-de-comercio/alquiler/rosario?pagina-1
[2023-12-17 11:48:41] - INFO - get_page - Obtendo código html da url https://www.argenprop.com/campos/alquiler/rosario?pagina-1

In [20]:
locais = list(dict.fromkeys([i['local'] for i in html]))

In [ ]:
_tipo = ['campos','casas','cocheras','departamentos','fondos-de-comercio','galpones','hoteles','locales','negocios-especiales','oficinas','ph','quintas','terrenos']
_local = ['rosario']

page_df = await ScraperArgenProp(_tipo = _tipo, _local = _local).get_property_data()

In [3]:
page_df.head()

,id,base,tipo_imovel,estado,cidade,bairro,endereco,url,descricao,titulo,...,tipo_local,imobiliaria,num_fotos,fotos,card_points,wsp,data,ano,mes,dia
0,12671111,argenprop,campos,santa fe,rosario,Alvear,autopista buenos aires km 283,https://www.argenprop.com/campo-en-alquiler-en...,Se alquilan 6 hectáreas juntas o fraccionadas ...,Campo - Alvear,...,Sem info,domum negocios inmobiliarios,9.0,[https://static1.sosiva451.com/11117621/dd79eb...,0.0,https://wa.me/5493416422361,2023-12-17 11:24:50,2023,12,17
1,10913192,argenprop,campos,santa fe,rosario,Perez,ruta 100,https://www.argenprop.com/campo-en-alquiler-en...,En alquiler fraccion de campo en perez co...,Excelente fracción en zona industrial de Pérez...,...,Sem info,casa lazio bienes raices,2.0,[https://static1.sosiva451.com/29131901/e36941...,0.0,Sem info,2023-12-17 11:24:50,2023,12,17
2,13238831,argenprop,casas,santa fe,rosario,Abasto,san martìn 2200,https://www.argenprop.com/casa-en-alquiler-en-...,La propiedad se ofrece solo para destino comer...,Alquiler - Casa 4 dormitorios (uso comercial) ...,...,Sem info,echeverria - massoud,47.0,[https://static1.sosiva451.com/13883231/093b26...,115.0,https://wa.me/5493416116361,2023-12-17 11:24:50,2023,12,17
3,14718440,argenprop,casas,santa fe,rosario,Abasto,dorrego 2200,https://www.argenprop.com/casa-en-alquiler-en-...,Casa de 2 plantas apto vivienda/comercial/inst...,Casa en Abasto,...,Sem info,Sem info,29.0,[https://static1.sosiva451.com/04481741/9b70c2...,0.0,https://wa.me/5493417191106,2023-12-17 11:24:56,2023,12,17
4,14753429,argenprop,casas,santa fe,rosario,Abasto,maipu 3400,https://www.argenprop.com/casa-en-alquiler-en-...,Casa de 2 dormitorios con fondo. Living comedo...,MAIPU 3400,...,Sem info,angueira miranda daniel inmobiliaria,62.0,[https://static1.sosiva451.com/92435741/ce5164...,0.0,https://wa.me/5493413902670,2023-12-17 11:24:56,2023,12,17


In [10]:
for i in range(len(page_df['cidade'].unique())):
    print(page_df['cidade'].unique()[i])

rosario


In [3]:
df_local = page_df[['id','cidade','bairro','endereco']].copy()

dict_locais = [
    {'id': x['id'],'endereco': f"{x['endereco']},{x['bairro']},{x['cidade']},santa fe,argentina"} 
    for index, x in df_local.iterrows()
]

In [ ]:
for i in range(len(dict_locais[:20])):
    latitude, longitude = geocoder.arcgis(dict_locais[i]['endereco']).json['lat'], geocoder.arcgis(dict_locais[i]['endereco']).json['lng']
    dict_locais[i]['lat'], dict_locais[i]['lon'] = latitude, longitude
    print(f'{dict_locais[i]["endereco"]} - Latitude: {latitude}, Longitude: {longitude}')
    time.sleep(1)

In [ ]:
dict_locais[:20]

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers = 10) as executor:
    # Criando a sequência de tasks que serão submetidas para a thread pool
    rows = {executor.submit(apply_geocoding, row): row for index, row in page_df.iterrows()}
    
    # # Loop para executar as tasks de forma concorrente. Também seria possível criar uma list comprehension que esperaria todos os resultados para retornar os valores.
    # for future in concurrent.futures.as_completed(rows):
    #     row = rows[future]
    #     try:
    #         resultado = future.result()
    #         res.append(resultado)
    #         # print(f'{url} OK')
    #     except Exception as exc:
    #         print(f'{url} com erro: {exc}')
            


In [ ]:
for index, row in page_df.iterrows():
    # 'row' is a Pandas Series where you can access columns using row['column']
    print(index, row)

In [ ]:
page_df2 = page_df

page_df2[['latitude', 'longitude']] = page_df2.apply(lambda row: apply_geocoding(row), axis=1)

In [3]:
get_geocoding(cidade = 'rosario', bairro = '', endereco = 'country aguadas 100')

[2023-12-14 21:36:50] - INFO - get_geocoding - Tratando nome da cidade!
[2023-12-14 21:36:51] - INFO - get_geocoding - Procurando estado para a cidade rosario - Argentina!
[2023-12-14 21:36:51] - DEBUG - _new_conn - Starting new HTTPS connection (1): geocode.arcgis.com:443
[2023-12-14 21:36:51] - DEBUG - _make_request - https://geocode.arcgis.com:443 "GET /arcgis/rest/services/World/GeocodeServer/find?f=json&text=country+aguadas+100%2Crosario%2Csanta+fe%2CArgentina&maxLocations=1 HTTP/1.1" 200 254
[2023-12-14 21:36:51] - INFO - _connect - Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=country+aguadas+100%2Crosario%2Csanta+fe%2CArgentina&maxLocations=1
[2023-12-14 21:36:51] - INFO - get_geocoding - Tentando obter coordenadas do local country aguadas 100,rosario,santa fe,Argentina!


{'address': 'A. Aguado 100, Rosario, S2013, Santa Fe',
 'bbox': {'northeast': [-32.911399611461, -60.687672100077],
  'southwest': [-32.913399611461, -60.689672100077]},
 'confidence': 9,
 'lat': -32.91239961146142,
 'lng': -60.68867210007655,
 'ok': True,
 'quality': 'StreetAddressExt',
 'raw': {'name': 'A. Aguado 100, Rosario, S2013, Santa Fe',
  'extent': {'xmin': -60.689672100077,
   'ymin': -32.913399611461,
   'xmax': -60.687672100077,
   'ymax': -32.911399611461},
  'feature': {'geometry': {'x': -60.68867210007655, 'y': -32.91239961146142},
   'attributes': {'Score': 91.53, 'Addr_Type': 'StreetAddressExt'}}},
 'score': 91.53,
 'status': 'OK'}

In [20]:
page_df[['cidade','bairro','endereco']].head(10)

,cidade,bairro,endereco
0,rosario,Alvear,autopista buenos aires km 283
1,rosario,Perez,ruta 100
2,rosario,Abasto,ocampo y 1 de mayo
3,rosario,Abasto,san martìn 2200
4,rosario,Abasto,dorrego 2200
5,rosario,Abasto,maipu 3400
6,rosario,Abasto,"virasoro , piso 00"
7,rosario,Aguadas,funes hills san marino 100
8,rosario,Aguadas,country aguadas 100
9,rosario,Aguadas,country aguadas 100


In [2]:
g = geocoder.arcgis('funes hills san marino 100,Aguadas,rosario,argentina')
g.json

[2023-12-17 11:14:08] - DEBUG - _new_conn - Starting new HTTPS connection (1): geocode.arcgis.com:443


[2023-12-17 11:14:09] - DEBUG - _make_request - https://geocode.arcgis.com:443 "GET /arcgis/rest/services/World/GeocodeServer/find?f=json&text=funes+hills+san+marino+100%2CAguadas%2Crosario%2Cargentina&maxLocations=1 HTTP/1.1" 200 255
[2023-12-17 11:14:09] - INFO - _connect - Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=funes+hills+san+marino+100%2CAguadas%2Crosario%2Cargentina&maxLocations=1


{'address': 'Deán Funes, Rosario, S2013, Santa Fe',
 'bbox': {'northeast': [-32.97457033894, -60.644410876862],
  'southwest': [-32.97657033894, -60.646410876862]},
 'confidence': 9,
 'lat': -32.97557033894,
 'lng': -60.64541087686172,
 'ok': True,
 'quality': 'StreetName',
 'raw': {'name': 'Deán Funes, Rosario, S2013, Santa Fe',
  'extent': {'xmin': -60.646410876862,
   'ymin': -32.97657033894,
   'xmax': -60.644410876862,
   'ymax': -32.97457033894},
  'feature': {'geometry': {'x': -60.64541087686172, 'y': -32.97557033894},
   'attributes': {'Score': 81.33, 'Addr_Type': 'StreetName'}}},
 'score': 81.33,
 'status': 'OK'}

In [2]:
print(get_state(cidade = 'rosario', pais = 'argentina'))

[2023-12-17 11:19:10] - INFO - get_state - Tratando nome da cidade!


[2023-12-17 11:19:11] - INFO - get_state - Procurando estado para a cidade rosario - argentina!


santa fe


In [13]:
# JSON COM DADOS DE MUNICIPIO/DEPARTAMENTO E PROVINCIA

df_mun = pd.read_parquet(
    os.path.join(os.getcwd(),'data','geograficos','provincias_municipios') if os.getcwd().__contains__('app') else os.path.join(os.getcwd(),'app','data','geograficos','provincias_municipios')
)
df_mun_final = df_mun[['municipio_tratado','provincia_tratada']].drop_duplicates()

df_dpt = pd.read_parquet(
    os.path.join(os.getcwd(),'data','geograficos','departamentos') if os.getcwd().__contains__('app') else os.path.join(os.getcwd(),'app','data','geograficos','departamentos')
)
df_dpt_final = df_dpt[['departamento', 'provincia']].drop_duplicates()

In [24]:
json_mun = {row['municipio_tratado']:row['provincia_tratada'] for index, row in df_mun_final.iterrows()}
json_dpt = {row['departamento']:row['provincia'] for index, row in df_dpt_final.iterrows()}

In [33]:
json_mun + json_dpt

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'